# Offline Features

Now that we have applied our features, let's use them to grab a training dataset consisting of the features we just defined.  
In this case we are fetching the full dataset with all the features, but as we will see in this notebook, it would be very simple to specify just a few features we wanted, take features from different FeatureViews, or a specific time-window that we are interested in.

In [ ]:
import feast
import pandas as pd
from datetime import datetime
import psycopg2
import yaml
import numpy as np

Just like before, we use our yaml file to initialize our Feast python client.

In [ ]:
with open('feature_repo/feature_store.yaml', 'r') as file:
    fs_config_yaml = yaml.safe_load(file)

fs_config = feast.repo_config.RepoConfig(**fs_config_yaml)
fs = feast.FeatureStore(config=fs_config)

### Point-in-time correctness
To get datapoints from Feast, we have to provide it with the ID we want to get features for as well as a date.  
Feast then finds the most recent feature values before that date for the selected features. This ensures that our models don’t accidentally use future information, preventing data leakage.  

In our case, we are using the song_rankings dataset to grab features from the song_properties data and merge them together.  
Because we have multiple songs from the same date and with the same ID in the rankings dataset, we are also adding a small date delta to make sure the Feast doesn't discard any of the duplicates.

In [ ]:
training_dataset = pd.read_parquet("../99-data_prep/song_rankings.parquet")
# Feast will remove rows with identical id and date so we add a small delta to each
microsecond_deltas = np.arange(0, len(training_dataset))*2
training_dataset['snapshot_date'] = training_dataset['snapshot_date'] + pd.to_timedelta(microsecond_deltas, unit='us')
training_dataset

Now we can specify what features we want to get, note that we also say what Feature View those features come from.

In [ ]:
features=[
        "song_properties:is_explicit",
        "song_properties:duration_ms",
        "song_properties:danceability",
        "song_properties:energy",
        "song_properties:key",
        "song_properties:loudness",
        "song_properties:mode",
        "song_properties:speechiness",
        "song_properties:acousticness",
        "song_properties:instrumentalness",
        "song_properties:liveness",
        "song_properties:valence",
        "song_properties:tempo"
]

And finally we fetch the historical data which will be our training dataset.

In [ ]:
training_df = fs.get_historical_features(entity_df=training_dataset, features=features).to_df()
training_df

We now know how to get training data from Feast, let's next look at how to use it during inference.  
In the next notebook we will fetch Online Features: [3-test_load_online_features.ipynb](3-test_load_online_features.ipynb)